In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.3.1
CUDA available: True
CUDA version: 12.1


In [2]:
# Check mmengine installation
import mmengine
print(f"MMengine: {mmengine.__version__}")

# Check Openmim installation
from importlib.metadata import version
print(f"Openmim: {version('openmim')}")

# Check mmcv installation
import mmcv
print("mmcv:",mmcv.__version__)

# Check MMDetection installation
import mmdet
print("mmdetection:",mmdet.__version__)

MMengine: 0.10.4
Openmim: 0.3.9
mmcv: 2.0.0rc4
mmdetection: 3.3.0


In [4]:
from pycocotools.coco import COCO

# Path to load the COCO annotation file
annotation_file = 'mmdetection/data/saf22/train/annotation_coco.json'

# Initialise the COCO object
coco = COCO(annotation_file)

# Get all category tags and corresponding category IDs
categories = coco.loadCats(coco.getCatIds())
category_id_to_name = {cat['id']: cat['name'] for cat in categories}

# Print all category IDs and corresponding category names
for category_id, category_name in category_id_to_name.items():
    print(f"Category ID: {category_id}, Category Name: {category_name}")

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Category ID: 1, Category Name: ClassicalEvaporation
Category ID: 2, Category Name: TansitionalMixing
Category ID: 3, Category Name: DiffusiveMixing
Category ID: 4, Category Name: spray


In [3]:
import mmcv
import mmengine
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
# Choose to use a config and initialize the detector
config_file = 'mmdetection/configs/htc/htc_x101-32x4d_fpn_16xb1-20e_coco.py'
# Setup a checkpoint file to load
checkpoint_file = 'mmdetection/configs/htc/htc_x101_32x4d_fpn_16x1_20e_coco_20200318-de97ae01.pth'

# register all modules in mmdet into the registries
register_all_modules()

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')  # or device='cuda:0'

Loads checkpoint by local backend from path: mmdetection/configs/htc/htc_x101_32x4d_fpn_16x1_20e_coco_20200318-de97ae01.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: roi_head.semantic_head.lateral_convs.0.conv.weight, roi_head.semantic_head.lateral_convs.0.conv.bias, roi_head.semantic_head.lateral_convs.1.conv.weight, roi_head.semantic_head.lateral_convs.1.conv.bias, roi_head.semantic_head.lateral_convs.2.conv.weight, roi_head.semantic_head.lateral_convs.2.conv.bias, roi_head.semantic_head.lateral_convs.3.conv.weight, roi_head.semantic_head.lateral_convs.3.conv.bias, roi_head.semantic_head.lateral_convs.4.conv.weight, roi_head.semantic_head.lateral_convs.4.conv.bias, roi_head.semantic_head.convs.0.conv.weight, roi_head.semantic_head.convs.0.conv.bias, roi_head.semantic_head.convs.1.conv.weight, roi_head.semantic_head.convs.1.conv.bias, roi_head.semantic_head.convs.2.conv.weight, roi_head.semantic_head.convs.2.conv.bias, roi_head.semantic

### Wandb integration

In [5]:
import wandb
wandb.login()

wandb: Currently logged in as: ahluwaliajyoti50 (ahluwaliajyoti50-university-of-sussex). Use `wandb login --relogin` to force relogin


True

In [6]:
# Weights and Biases version
print(wandb.__version__)

0.17.5


# Medium with epoch size - 500

## Modify the config

In the next step, we need to modify the config for the training. To accelerate the process, we finetune a detector using a pre-trained detector.

In [7]:
from mmengine.runner import set_random_seed
from mmengine import Config
import random
import numpy as np
import matplotlib.pyplot as plt
from mmdet.visualization import DetLocalVisualizer
import os
import cv2

In [8]:
# Load the configuration
cfg = Config.fromfile('mmdetection/configs/htc/htc_x101-32x4d_fpn_16xb1-20e_coco.py')

# Modify dataset classes and color
cfg.metainfo = {
    'classes': ('ClassicalEvaporation', 'TansitionalMixing', 'DiffusiveMixing', 'spray'),
    'palette': [
        (13,24,103), (167,13,13), (91,117,249), (203,173,55),
    ]
}

# Modify dataset type and path
cfg.data_root = 'mmdetection/data/saf22/'
cfg.train_dataloader.dataset.ann_file = 'train/annotation_coco.json'
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = {'img': 'train/'}
cfg.train_dataloader.dataset.metainfo = cfg.metainfo

cfg.val_dataloader.dataset.ann_file = 'val/annotation_coco.json'
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = {'img': 'val/'}
cfg.val_dataloader.dataset.metainfo = cfg.metainfo

cfg.test_dataloader = cfg.val_dataloader

# Modify metric config
cfg.val_evaluator.ann_file = f'{cfg.data_root}val/annotation_coco.json'
cfg.test_evaluator = cfg.val_evaluator

################################################################################################

# Modify num classes of the model in box head and mask head
if isinstance(cfg.model.roi_head.bbox_head, list):
    for head in cfg.model.roi_head.bbox_head:
        head.num_classes = 4
else:
    cfg.model.roi_head.bbox_head.num_classes = 4

if isinstance(cfg.model.roi_head.mask_head, list):
    for head in cfg.model.roi_head.mask_head:
        head.num_classes = 4
else:
    cfg.model.roi_head.mask_head.num_classes = 4

################################################################################################

# Configure checkpoint saving
cfg.default_hooks.checkpoint = dict(
    type='CheckpointHook',
    interval=1,
    save_optimizer=True,
    save_best = ['coco/bbox_mAP','coco/segm_mAP'],
    save_last=True,
    rule='greater',
    max_keep_ckpts=1
)

cfg.test_evaluator.update(dict(classwise=True))

# Use the pre-trained HTC model
cfg.load_from = 'mmdetection/configs/htc/htc_x101_32x4d_fpn_16x1_20e_coco_20200318-de97ae01.pth'

# Set up working dir to save files and logs
cfg.work_dir = './tutorial_exps/500-epochs-batch1'

# Set the number of epochs
cfg.train_cfg.max_epochs = 500

# Set the evaluation interval
cfg.train_cfg.val_interval = 3

# Set the checkpoint saving interval to reduce the storage cost
cfg.default_hooks.checkpoint.interval = 3

# Adjust learning rate for single GPU
cfg.optim_wrapper.optimizer.lr = 0.02 / 8

# Set logging interval
cfg.default_hooks.logger.interval = 10

# Set seed for reproducibility
set_random_seed(0, deterministic=False)

# Use Tensorboard to log the training process
cfg.visualizer.vis_backends.append({"type": 'TensorboardVisBackend'})

# Configure WandB logger
cfg.visualizer.vis_backends.append(dict(type='WandbVisBackend', init_kwargs=dict(
    project='SAF_Project',
    entity='ahluwaliajyoti50-university-of-sussex',
    config={
        'learning_rate': cfg.optim_wrapper.optimizer.lr,
        'architecture': 'HTC_resnext101_32x4d',
        'dataset': 'SAF22',
        'epochs': cfg.train_cfg.max_epochs,
        'batch_size': cfg.train_dataloader.batch_size,
        'optimizer': cfg.optim_wrapper.optimizer.type,
        'classes': cfg.metainfo['classes'],
        'seed': 0,
    }
)))

# Save the modified config
config = 'mmdetection/configs/htc/htc_x101-32x4d_fpn_16xb1-20e_coco_saf-500-batch1.py'
with open(config, 'w') as f:
    f.write(cfg.pretty_text)

# Train using train.py
%run mmdetection/tools/train.py {config}

# [Optional] Finish the wandb run
wandb.finish()

08/08 11:55:08 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.10.14 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:44:50) [MSC v.1916 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 209652396
    GPU 0: NVIDIA RTX A4000
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1
    NVCC: Cuda compilation tools, release 12.1, V12.1.66
    MSVC: n/a, reason: fileno
    PyTorch: 2.3.1
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 192930154
  - Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=comput

08/08 11:55:22 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
08/08 11:55:22 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

c:\users\ja683\downloads\saf\mmdetection\mmdet\models\roi_heads\mask_heads\fcn_mask_head.py:339: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  mask_preds = bboxes.new_tensor(mask_preds)


08/08 11:57:20 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.045
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.071
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.044
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.650
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.816
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.272
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.272
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.272
 Average

coco/bbox_mAP,▁▃▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇▇▇███████
coco/bbox_mAP_50,▁▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████████████████
coco/bbox_mAP_75,▁▅▆▆▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████▇████▇██▇
coco/bbox_mAP_l,█▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
coco/bbox_mAP_m,█████▂██▆▆▆▆▆▆▆▆▁▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
coco/bbox_mAP_s,▁▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████
coco/segm_mAP,▁▆▅▆▆▆▆▆▆▆███▆█▆▇████████▆▆▆▇▇▆▆▆▆▆▆▆▆▆▆
coco/segm_mAP_50,▁▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
coco/segm_mAP_75,▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████▇██▇█▇▇▇▇▇▇▇▇
coco/segm_mAP_l,▁▃▁▃▃▃▃▃▃▃███▃█▃▄████████▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
coco/segm_mAP_m,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [9]:
config = 'mmdetection/configs/htc/htc_x101-32x4d_fpn_16xb1-20e_coco_saf-500-batch1.py'

In [10]:
from glob import glob
ckpt = 'tutorial_exps/500-epochs-batch1/best_coco_segm_mAP_epoch_261.pth'

In [11]:
ckpt_b  = 'tutorial_exps/500-epochs-batch1/best_coco_bbox_mAP_epoch_345.pth'

## Test HTC on best checkpointfile and save the painted images for future visualization 

In [12]:
!python mmdetection/tools/test.py {config} {ckpt} --show-dir output/segm

08/08 14:15:41 - mmengine - 

wandb: Currently logged in as: ahluwaliajyoti50 (ahluwaliajyoti50-university-of-sussex). Use `wandb login --relogin` to force relogin


INFO

wandb: wandb version 0.17.6 is available!  To upgrade, please run:

 - 

...

wandb: Find logs at: .\tutorial_exps\500-epochs-batch1\20240808_141535\vis_data\wandb\run-20240808_141544-4u31inqj\logs


  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90


wandb: WARNING The new W&B backend becomes opt-out in version 0.18.0; try it out with `wandb.require("core")`! See https://wandb.me/wandb-core for more information.

  - CuDNN 8.9.7  (built against CUDA 12.2)


  - Magma 2.5.4
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.7, CXX_COMPILER=C:/cb/pytorch_1000000000000/work/tmp_bin/sccache-cl.exe, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /Zc:__cplusplus /bigobj /FS /utf-8 -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINETO -DLIBKINETO_NOCUPTI -DLIBKINETO_NOROCTRACER -DUSE_FBGEMM -DUSE_XNNPACK -DSYMBOLICATE_MOBILE_DEBUG_HANDLE /wd4624 /wd4068 /wd4067 /wd4267 /wd4661 /wd4717 /wd4244 /wd4804 /wd4273, LAPACK_INFO=mkl, PERF_WITH_AVX=1, PERF_WITH_AVX2=1, PERF_WITH_AVX512=1, TORCH_VERSION=2.3.1, USE_CUDA=ON, USE_CUDNN=ON, USE_CUSPARSELT=OFF, USE_EXCEPTION_PTR=1, USE_GFLAGS=OFF, USE_GLOG=OFF, USE_GLOO=ON, USE_MKL=ON, USE_MKLDNN=ON, USE_MPI=OFF, USE_NCCL=OFF, USE_NNPACK=OFF, USE_OPENMP=ON, USE_ROCM=OFF, USE_ROCM_KERNEL_ASSERT=OFF, 

    TorchVision: 0.18.1
    OpenCV: 4.10.0
    MMEngine: 0.10.4

Runtime environment:
    cudnn_benchmark: False
    dist_cfg: {'backend': 'nccl'}
    mp_cfg: {'mp_start_method': 'fork', 'openc

## Inference on trained Model

In [13]:
import os
from mmdet.apis import init_detector, inference_detector
import mmcv
import cv2
from mmdet.registry import VISUALIZERS

model = init_detector(config, ckpt, device='cuda:0')

image_folder = 'mmdetection/data/saf22/test/'
output_folder = 'tutorial_exps/500-epochs-batch1/inference-results/segm'
image_paths = [os.path.join(image_folder, img_name) for img_name in os.listdir(image_folder) if img_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff'))]

visualizer = VISUALIZERS.build(model.cfg.visualizer)
visualizer.dataset_meta = model.dataset_meta

Loads checkpoint by local backend from path: tutorial_exps/500-epochs-batch1/best_coco_segm_mAP_epoch_261.pth


C:\Users\ja683\AppData\Roaming\Python\Python310\site-packages\mmengine\utils\manager.py:113: UserWarning: <class 'mmdet.visualization.local_visualizer.DetLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


In [14]:
for img_path in image_paths:
    img = mmcv.imread(img_path)
    if img is None:
        print(f"Failed to read image at {img_path}")
        continue
    
    new_result_segm = inference_detector(model, img)
    os.makedirs(output_folder, exist_ok=True)
    output_file = os.path.join(output_folder, os.path.basename(img_path))
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    visualizer.add_datasample(
        'new_result_segm',
        img_rgb,
        data_sample=new_result_segm,        draw_gt=None,
        wait_time=0,
        out_file=output_file
    )

## Save results.pkl

In [15]:
!python mmdetection/tools/test.py {config} {ckpt}  --out ./results/500-epochs-batch1/results.pkl

08/08 14:19:42 - mmengine - 

wandb: Currently logged in as: ahluwaliajyoti50 (ahluwaliajyoti50-university-of-sussex). Use `wandb login --relogin` to force relogin


INFO - 
------------------------------------------------------------

wandb: wandb version 0.17.6 is available!  To upgrade, please run:

System environment:


...

wandb: Find logs at: .\tutorial_exps\500-epochs-batch1\20240808_141937\vis_data\wandb\run-20240808_141944-ouoypzfo\logs


  - CUDA Runtime 12.1


wandb: WARNING The new W&B backend becomes opt-out in version 0.18.0; try it out with `wandb.require("core")`! See https://wandb.me/wandb-core for more information.

  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90


  - CuDNN 8.9.7  (built against CUDA 12.2)
  - Magma 2.5.4
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.7, CXX_COMPILER=C:/cb/pytorch_1000000000000/work/tmp_bin/sccache-cl.exe, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /Zc:__cplusplus /bigobj /FS /utf-8 -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINETO -DLIBKINETO_NOCUPTI -DLIBKINETO_NOROCTRACER -DUSE_FBGEMM -DUSE_XNNPACK -DSYMBOLICATE_MOBILE_DEBUG_HANDLE /wd4624 /wd4068 /wd4067 /wd4267 /wd4661 /wd4717 /wd4244 /wd4804 /wd4273, LAPACK_INFO=mkl, PERF_WITH_AVX=1, PERF_WITH_AVX2=1, PERF_WITH_AVX512=1, TORCH_VERSION=2.3.1, USE_CUDA=ON, USE_CUDNN=ON, USE_CUSPARSELT=OFF, USE_EXCEPTION_PTR=1, USE_GFLAGS=OFF, USE_GLOG=OFF, USE_GLOO=ON, USE_MKL=ON, USE_MKLDNN=ON, USE_MPI=OFF, USE_NCCL=OFF, USE_NNPACK=OFF, USE_OPENMP=ON, USE_ROCM=OFF, USE_ROCM_KERNEL_ASSERT=OFF, 

    TorchVision: 0.18.1
    OpenCV: 4.10.0
    MMEngine: 0.10.4

Runtime environment:
    cudnn_benchmark: False
    dist_cfg: {'backend': 'nccl'}
   

The masks are represented as a tensor of boolean values, these boolean tensors represent the masks directly, rather than as coordinates.

## Log Analysis

In [16]:
!pip show matplotlib

Name: matplotlib
Version: 3.9.1
Summary: Python plotting package
Home-page: 
Author: John D. Hunter, Michael Droettboom
Author-email: Unknown <matplotlib-users@python.org>
License: License agreement for matplotlib versions 1.3.0 and later
        
        1. This LICENSE AGREEMENT is between the Matplotlib Development Team
        ("MDT"), and the Individual or Organization ("Licensee") accessing and
        otherwise using matplotlib software in source or binary form and its
        associated documentation.
        
        2. Subject to the terms and conditions of this License Agreement, MDT
        hereby grants Licensee a nonexclusive, royalty-free, world-wide license
        to reproduce, analyze, test, perform and/or display publicly, prepare
        derivative works, distribute, and otherwise use matplotlib
        alone or in any derivative version, provided, however, that MDT's
        License Agreement and MDT's notice of copyright, i.e., "Copyright (c)
        2012- Matplot

In [17]:
!pip install matplotlib

In [18]:
import matplotlib
print(matplotlib.rcParams['backend'])

module://matplotlib_inline.backend_inline


In [19]:
json_log = 'tutorial_exps/500-epochs-batch1/20240808_115501/vis_data/20240808_115501.json'

In [20]:
!python mmdetection/tools/analysis_tools/analyze_logs.py plot_curve \
  --keys loss \
  --title "Training Metrics" \
  --legend "Loss" \
  --backend TkAgg \
  --style darkgrid \
  --out tutorial_exps/500-epochs-batch1/20240808_115501/training_curves.png \
  {json_log}

plot curve of tutorial_exps/500-epochs-batch1/20240808_115501/vis_data/20240808_115501.json, metric is loss
save curve to: tutorial_exps/500-epochs-batch1/20240808_115501/training_curves.png


### Compute the average training speed.


In [21]:
!python mmdetection/tools/analysis_tools/analyze_logs.py cal_train_time {json_log} --include-outliers

-----Analyze train time of tutorial_exps/500-epochs-batch1/20240808_115501/vis_data/20240808_115501.json-----
slowest epoch 1, average time is 3.3532 s/iter
fastest epoch 2, average time is 0.3356 s/iter
time std over epochs is 0.1333
average iter time: 0.3832 s/iter



## Result Analysis


In [22]:
!python mmdetection/tools/analysis_tools/analyze_results.py \
       {config} \
       ./results/500-epochs-batch1/results.pkl \
       ./results/500-epochs-batch1/vis_data \
       --show
# [Optional] Finish the wandb run
wandb.finish()

wandb: Currently logged in as: ahluwaliajyoti50 (ahluwaliajyoti50-university-of-sussex). Use `wandb login --relogin` to force relogin

loading annotations into memory...



wandb: wandb version 0.17.6 is available!  To upgrade, please run:

Done (t=0.02s)


creating index...

...

------------------------------------------------------------



wandb:  View project at https://wandb.ai/ahluwaliajyoti50-university-of-sussex/SAF_Project


System environment:

wandb:  View run at https://wandb.ai/ahluwaliajyoti50-university-of-sussex/SAF_Project/runs/uryzcehi

    sys.platform: win32
    Python: 3.10.14 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:44:50) [MSC v.1916 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1927330238
    GPU 0: NVIDIA RTX A4000
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1
    NVCC: Cuda compilation tools, release 12.1, V12.1.66
    MSVC: Microsoft (R) C/C++ Optimizing Compiler Version 19.35.32216.1 for x64
    GCC: n/a
    PyTorch: 2.3.1
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 192930154
  - Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-

## Confusion Matrix

In [23]:
# [Optional] Finish the wandb run
wandb.finish()
!python mmdetection/tools/analysis_tools/confusion_matrix.py \
        {config} \
        results/500-epochs-batch1/results.pkl \
        results/500-epochs-batch1 --show

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
[                                                  ] 0/8, elapsed: 0s, ETA:
[>>>>                             ] 1/8, 334.8 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>                         ] 2/8, 501.7 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>                     ] 3/8, 602.0 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>                 ] 4/8, 668.7 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>             ] 5/8, 835.9 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>        ] 6/8, 1003.1 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 7/8, 1003.4 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 1146.7 task/s, elapsed: 0s, ETA:     0sFigure(1500x956.25)


In [24]:
!pip install scikit-learn matplotlib seaborn

## Eval Metric from results.pkl file

In [25]:
!python mmdetection/tools/analysis_tools/eval_metric.py \
        {config} \
        results/500-epochs-batch1/results.pkl

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
08/08 14:28:49 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.06s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.294
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.356
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.331
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.850
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.900
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.362
 Av